In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import nltk
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train.txt", delimiter=';', header=None, names=['sentence','label'])
test = pd.read_csv("test.txt", delimiter=';', header=None, names=['sentence','label'])
val = pd.read_csv("val.txt", delimiter=';', header=None, names=['sentence','label'])

In [4]:
df_data = pd.concat([train, test,val])

df_data

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [5]:
df_data.to_csv (r'exportdata.txt', index=False)
dt_data =  pd.read_csv("exportdata.txt")

dt_data

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
19995,im having ssa examination tomorrow in the morn...,sadness
19996,i constantly worry about their fight against n...,joy
19997,i feel its important to share this info for th...,joy
19998,i truly feel that if you are passionate enough...,joy


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_data, 
                                                    df_data['label'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

In [11]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [13]:
from sklearn.pipeline import Pipeline

In [14]:
#FE
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer


# cv = CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer = token.tokenize)
# text = cv.fit_transform(dt_data['sentence'])

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
sentence = Pipeline([
                ('imputer', TextImputer('sentence', '')),
                ('selector', ColumnSelector(key='sentence')),
                ('CountVectorizer', CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer = token.tokenize))
            ])


# feats = FeatureUnion([('description', description),
#                       ('company_profile', company_profile),
#                       ('benefits', benefits)])

In [15]:
%%time
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('sentence', sentence),
    ('classifier', MultinomialNB()),
])

pipeline.fit(X_train, y_train)

Wall time: 201 ms


C:\Users\ALIENWARE\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Pipeline(steps=[('sentence',
                 Pipeline(steps=[('imputer',
                                  TextImputer(key='sentence', value='')),
                                 ('selector', ColumnSelector(key='sentence')),
                                 ('CountVectorizer',
                                  CountVectorizer(stop_words='english',
                                                  tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>))])),
                ('classifier', MultinomialNB())])

In [16]:
pipeline.steps

[('sentence',
  Pipeline(steps=[('imputer', TextImputer(key='sentence', value='')),
                  ('selector', ColumnSelector(key='sentence')),
                  ('CountVectorizer',
                   CountVectorizer(stop_words='english',
                                   tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>))])),
 ('classifier', MultinomialNB())]

In [26]:
import dill

In [27]:
with open("multinomialNB_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [1]:
!python --version

Python 3.7.9
